In [1]:
import urllib.request
import collections
import math
import os
import random
import zipfile
import datetime as dt
import numpy as np
import tensorflow as tf
import pandas as pd
import nltk
import re
from string import punctuation
import matplotlib.pyplot as plt
from nltk.util import ngrams
from itertools import chain


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

def build_dataset(words, n_words): # n_words is the size of the vocabulary that we want taken as 10000 here
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]] # initialise a list that counts the occurance of each word
    count.extend(collections.Counter(words).most_common(n_words - 1)) # fill the list with each word count as a tuple (word, word_count)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary) # create a dictionary where keys are the words and their values are their rank of counts eg. as:1 if as is ranked 1 in the count list
    data = list()
    unk_count = 0
    # the created index for each word (based on its count rank) is now stored in the list named "data"
    for word in words:
        if word in dictionary:
            index = dictionary[word] # index of the word being initialised as per the count of the word
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index) # contains index of each word based on its count rank 
                        # BUT NOTE THE INDEX ARE NOT ORDERED AS PER COUNTS AND PRESERVE THEIR ORIGNINAL ORDER IN THE SENTENCES
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def collect_data(vocabulary_size=13648):
    filename = 'filename.zip'
    vocabulary = read_data(filename) #returns a list of all words in the text file
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size) # Provide the list of words and vocabulary size to create the dataset
    del vocabulary  # Hint to reduce memory.
    # the data contains list of indexes of each word as per their occurence sequence in the text file
    # Try printing print(reverse_dictionary[5234],reverse_dictionary[3081],reverse_dictionary[12],reverse_dictionary[6],reverse_dictionary[195]) 
    
    # the count is a list of tuples (word, word_count)
    # dictionary contains keys as words and their index as values
    # dictionary contains keys as index and their corresponding words as values
    
    return data, count, dictionary, reverse_dictionary

data_index = 0 # initialize data index and maake it global to use in the function bellow
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0 # proceed only if the condition is true " BATCH SIZE IS A MULTIPLE OF num_skips"
    assert num_skips <= 2 * skip_window # number of words to be randomly picked from surrounding the context word should be less than equal to skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32) # initialize a nd.array for the batch of input with size (batch_size,) eg. (128,)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32) # initialize a nd.array for the context with size (batch_size,1) 
                    # eg. (128, 1) to hold index of context words for each word (index) in batch array([[  18],[  18],[   1],[   1],..])
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span) # A double ended queu where elements can be added or subtracted from both ends
    for _ in range(span):
        buffer.append(data[data_index]) # data_index is initiallized by 0 outsuide the function
        data_index = (data_index + 1) % len(data) # after this step in the buffer 5 words are stored based on their index in the supplied data list
                            # The data list contains index of each word in the order of sequence they occur in the text and thus it preserve the order of the context 
    for i in range(batch_size // num_skips): # batch_size divided by num_skips =128/2 = 64
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1) #generates a random integer between 0-4 so that the index of the words surrounding the input word can be obtained
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word ## at the between of the buffer
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index]) # find the comment bellow
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

'''
This buffer will hold a maximum of span elements and will be a kind of moving window of words that samples are drawn from. 
Whenever a new word index is added to the buffer, the left most element will drop out of the buffer to allow room for the new 
word index being added.  The position of the buffer in the input text stream is stored in a global variable data_index which is
incremented each time a new word is added to the buffer.  If it gets to the end of the text stream, the “% len(data)” component 
of the index update will basically reset the count back to zero.
'''

vocabulary_size=13648
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=13648)

['during', 'trip', 'operation', 'damage', 'wa', 'observe', 'on']


In [3]:
batch_size = 128
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 5       # How many words to consider left and right.
num_skips = 0         # How many times to reuse an input to generate a label. 

## By reading the blog the understanding of num_skips is that this parameter suggests the
## number of words to be randomly picked from the words in the context neighborhood of 'sat'

'''
For instance, in the 5-gram “the cat sat on the”, the input word will be center word i.e. “sat” and 
the context words that will be predicted will be drawn randomly from the remaining words of the gram: 
[‘the’, ‘cat’, ‘on1’, ‘the’].  In this function, the number of words drawn randomly from the surrounding 
context is defined by the argument num_skips.  The size of the window of context words to draw from around'
the input word is defined in the argument skip_window – in the example above (“the cat sat on the”), we have 
a skip window width of 2 around the input word “sat”.
'''

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 15    # Number of negative examples to sample.

In [4]:
# getting word embeddings from google
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

#again, download and load the model
model_gigaword = api.load("glove-wiki-gigaword-100")

In [5]:
with open("glove-wiki-gigaword-100.vocab", "w", encoding='utf-8') as f:
        for _, w in sorted((voc.index, word) for word, voc in model_gigaword.vocab.items()):
            print(w, file=f)

In [ ]:
with open("glove-wiki-gigaword-100.vocab", encoding='utf-8') as f:
    vocab_list = map(str.strip, f.readlines())
    
vocab_dict = {w: k for k, w in enumerate(vocab_list)}
vocab_dict

In [7]:
embedding_google = model_gigaword.syn0
np.save('Embeddings_generated_Google_25_MAY', embedding_google)

C:\Users\warlo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [8]:
#Saving the dictionary
import json
json = json.dumps(vocab_dict)
f = open("word_index_dictionary_for_Google_embeddings_25_MAY.json","w")
f.write(json)
f.close()

In [4]:
from nltk.stem import WordNetLemmatizer 
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
wnl = WordNetLemmatizer()
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'    
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings. 
    words_lement = [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(word_tokenize(text))]
    combined_string = ' '.join(map(str, words_lement))
    return  combined_string # ' '.join([for words in words_lement]) #' '.join([wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(word_tokenize(text))])

lementize_words =  lambda x: lemmatize_sent(str(x))
# fail_mech['lementized_form_words'] = fail_mech['Tokens'].apply(lementize_words)
# fail_mech

In [6]:
embeddings_google= np.load('Embeddings_generated_Google_25_MAY.npy')
norm_google=np.sqrt(np.sum(np.square(embeddings_google), axis=1))
embeddings_google_scaled=embeddings_google/norm_google[:,None]
import json
with open('word_index_dictionary_for_Google_embeddings_25_MAY.json', 'r') as fp:
    vocab_dict = json.load(fp)

In [ ]:
import json
with open('equipment_dictionary.json', 'r') as fp:
    dictionary_mp_eqpt = json.load(fp)
for key in dictionary_mp_eqpt.keys():
    dictionary_mp_eqpt[key].append(key)
dictionary_mp_eqpt

dictionary_mp_eqpt['tool_control'].append('programmable_logic_control')
dictionary_mp_eqpt['motor_cooling_system'].append('duct')
dictionary_mp_eqpt['motor_cooling_system'].append('air_duct')
dictionary_mp_eqpt['motor_cooling_system'].append('arrestor')
rem_ = ['fastener','seal']
for v in dictionary_mp_eqpt.values():
    for r_v in rem_:
        if r_v in v:
            v.remove(r_v)
dictionary_mp_eqpt  

In [ ]:
ept_taxo_dict = {}
for key, value in dictionary_mp_eqpt.items():
    for val in value:
        if val in [*vocab_dict.keys()]:
            print(key, val)
            print(vocab_dict[val])
            ept_taxo_dict.setdefault(key,[]).append(val)            

In [9]:
from itertools import combinations
eqpt_taxo_dict_combs = {}
for key, value in ept_taxo_dict.items():
    eqpt_taxo_dict_combs.setdefault(key,[]).append(list(combinations(value,2)))

In [ ]:
eqpt_taxo_dict_combs_synonyms = {}
for key, value in eqpt_taxo_dict_combs.items():
    for val1 in value:
        for val2 in val1:
            print(val2)
            eqpt_taxo_dict_combs_synonyms.setdefault(key,[]).append(val2)

In [ ]:
from itertools import product
all_val_flat_list = [item for sublist in [*ept_taxo_dict.values()] for item in sublist]
eqpt_taxo_dict_combs_antonyms = {}
for key, value in ept_taxo_dict.items():
    rem_val = [item for item in all_val_flat_list if item not in value]
    eqpt_taxo_dict_combs_antonyms.setdefault(key,[]).append(list(product(value,rem_val)))
eqpt_taxo_dict_combs_antonyms

In [ ]:
eqpt_taxo_dict_combs_antonyms_ = {}
for key, value in eqpt_taxo_dict_combs_antonyms.items():
    for val1 in value:
        for val2 in val1:
            print(val2)
            eqpt_taxo_dict_combs_antonyms_.setdefault(key,[]).append(val2)
eqpt_taxo_dict_combs_antonyms_

In [ ]:
all_synonym_lst = [item for sublist in [*eqpt_taxo_dict_combs_synonyms.values()] for item in sublist]
all_synonym_lst

In [ ]:
# this list would be the one from which we would extract negative pairs
comparison_lst_ =[item for sublist in [*ept_taxo_dict.values()] for item in sublist]
comparison_lst_

In [18]:
# embeddings_google= np.load('Embeddings_generated_Google_25_MAY.npy')
# norm_google=np.sqrt(np.sum(np.square(embeddings_google), axis=1))
# embeddings_google_scaled=embeddings_google/norm_google[:,None]

In [ ]:
all_antonym_lst = [item for sublist in [*eqpt_taxo_dict_combs_antonyms.values()] for item in sublist]
all_antonym_lst = sum(all_antonym_lst, [])
all_antonym_lst

In [17]:
'''
Idea is to create a subset of embeddings to look up from and store the trained vectors in other subset
forward and reverse dictionary is to be created for the only for the tokens of interest 
'''
flat_values = [item for sublist in [*ept_taxo_dict.values()] for item in sublist]

fwd_subset = dict((k, vocab_dict[k]) for k in flat_values if k in vocab_dict)
bwd_subset  = {v: k for k, v in fwd_subset.items()}

In [35]:
from scipy.spatial.distance import cosine
comparison_lst_
comparison_indices_ = [vocab_dict[w] for w in comparison_lst_]
comparison_indices_
def extract_negative(embeddings, list_indices, syn_indicator_):
    if syn_indicator_ ==True:
        #embed_subset = tf.nn.embedding_lookup(embeddings, list_indices) #subsetting the embeddings for the words to cluster
        #normalized = tf.nn.l2_normalize(embed_subset, dim = 1)
        negative_examples_synonyms = []
        for val in list_indices:
            lst_comb= [val]
            lst_1 = [w1 for w1 in comparison_indices_ if w1 not in lst_comb]
            embedding_array_ = []
            for w in lst_1:
                #embedding_array_.append(tf.nn.embedding_lookup(embeddings, w))
                embedding_array_.append(embeddings[w])
            embedding_val = embeddings[val]
            cos_lst = [cosine(embd_, embedding_val) for embd_ in embedding_array_]
            indx_ = np.argmin(cos_lst)
            neg_ = lst_1[indx_]
            negative_examples_synonyms.append(neg_)
        return negative_examples_synonyms
    else:
        negative_examples_antonyms = []
        for val in list_indices:
            lst_comb= [val]
            lst_1 = [w1 for w1 in comparison_indices_ if w1 not in lst_comb]
            embedding_array_ = []
            for w in lst_1:
                embedding_array_.append(embeddings[w])
            embedding_val = embeddings[val]
            cos_lst = [cosine(embd_, embedding_val) for embd_ in embedding_array_]
            indx_ = np.argmax(cos_lst)
            neg_ = lst_1[indx_]
            negative_examples_antonyms.append(neg_)
        return negative_examples_antonyms

In [36]:
graph = tf.Graph()
with graph.as_default():
    """
    Initialises the TensorFlow Attract-Repel model.
    """
    attract_examples_lt = tf.placeholder(tf.int32, shape=(None)) # each element is the position of word vector.
    attract_examples_rt = tf.placeholder(tf.int32, shape=(None)) # each element is the position of word vector.
    
    repel_examples_lt = tf.placeholder(tf.int32, shape=(None)) # each element is again the position of word vector.
    repel_examples_rt = tf.placeholder(tf.int32, shape=(None)) # each element is again the position of word vector.
    
    negative_examples_attract_lt = tf.placeholder(tf.int32, shape=(None))
    negative_examples_attract_rt = tf.placeholder(tf.int32, shape=(None))
    
    negative_examples_repel_lt = tf.placeholder(tf.int32, shape=(None))
    negative_examples_repel_rt = tf.placeholder(tf.int32, shape=(None))
    
    attract_margin = tf.placeholder("float")
    repel_margin = tf.placeholder("float")
    regularisation_constant = tf.placeholder("float")

    # Initial (distributional) vectors. Needed for L2 regularisation.         
    W_init = tf.constant(embeddings_google_scaled, name="W_init")

    # Variable storing the updated word vectors. 
    W_dynamic = tf.Variable(embeddings_google_scaled, name="W_dynamic")
    
    # Attract Cost Function: 

    # placeholders for example pairs...
    attract_examples_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, attract_examples_lt), 1) 
    attract_examples_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, attract_examples_rt), 1)

    # and their respective negative examples:
    negative_examples_attract_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_attract_lt), 1)
    negative_examples_attract_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_attract_rt), 1)

    # dot product between the example pairs. 
    attract_similarity_between_examples = tf.reduce_sum(tf.multiply(attract_examples_left, attract_examples_right), 1) 

    # dot product of each word in the example with its negative example. 
    attract_similarity_to_negatives_left = tf.reduce_sum(tf.multiply(attract_examples_left, negative_examples_attract_left), 1) 
    attract_similarity_to_negatives_right = tf.reduce_sum(tf.multiply(attract_examples_right, negative_examples_attract_right), 1)

    # and the final Attract Cost Function (sans regularisation):
    attract_cost = tf.nn.relu(attract_margin + attract_similarity_to_negatives_left - attract_similarity_between_examples) + \
                   tf.nn.relu(attract_margin + attract_similarity_to_negatives_right - attract_similarity_between_examples)

    # Repel Cost Function: 

    # placeholders for example pairs...
    repel_examples_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, repel_examples_lt), 1) # becomes batch_size X vector_dimension 
    repel_examples_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, repel_examples_rt), 1)

    # and their respective negative examples:
    negative_examples_repel_left  = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_repel_lt), 1)
    negative_examples_repel_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_repel_rt), 1)

    # dot product between the example pairs. 
    repel_similarity_between_examples = tf.reduce_sum(tf.multiply(repel_examples_left, repel_examples_right), 1) # becomes batch_size again, might need tf.squeeze

    # dot product of each word in the example with its negative example. 
    repel_similarity_to_negatives_left = tf.reduce_sum(tf.multiply(repel_examples_left, negative_examples_repel_left), 1)
    repel_similarity_to_negatives_right = tf.reduce_sum(tf.multiply(repel_examples_right, negative_examples_repel_right), 1)

    # and the final Repel Cost Function (sans regularisation):
    repel_cost = tf.nn.relu(repel_margin - repel_similarity_to_negatives_left + repel_similarity_between_examples) + \
                   tf.nn.relu(repel_margin - repel_similarity_to_negatives_right + repel_similarity_between_examples)

    # The Regularisation Cost (separate for the two terms, depending on which one is called): 

    # load the original distributional vectors for the example pairs: 
    original_attract_examples_left = tf.nn.embedding_lookup(W_init, attract_examples_lt)
    original_attract_examples_right = tf.nn.embedding_lookup(W_init, attract_examples_rt)

    original_repel_examples_left = tf.nn.embedding_lookup(W_init, repel_examples_lt)
    original_repel_examples_right = tf.nn.embedding_lookup(W_init, repel_examples_rt)

    # and then define the respective regularisation costs:
    regularisation_cost_attract = regularisation_constant * (tf.nn.l2_loss(original_attract_examples_left - attract_examples_left) + tf.nn.l2_loss(original_attract_examples_right - attract_examples_right))
    attract_cost += regularisation_cost_attract

    regularisation_cost_repel = regularisation_constant * (tf.nn.l2_loss(original_repel_examples_left - repel_examples_left) + tf.nn.l2_loss(original_repel_examples_right - repel_examples_right))
    repel_cost += regularisation_cost_repel

    # Finally, we define the training step functions for both steps. 

    tvars = tf.trainable_variables()
    attract_grads = [tf.clip_by_value(grad, -2., 2.) for grad in tf.gradients(attract_cost, tvars)]
    repel_grads = [tf.clip_by_value(grad, -2., 2.) for grad in tf.gradients(repel_cost, tvars)]

    attract_optimiser = tf.train.AdagradOptimizer(0.05) 
    repel_optimiser = tf.train.AdagradOptimizer(0.05) 

    attract_cost_step = attract_optimiser.apply_gradients(zip(attract_grads, tvars))
    repel_cost_step = repel_optimiser.apply_gradients(zip(repel_grads, tvars))
    init = tf.global_variables_initializer()
    

In [37]:
import time
import random
def run(graph, all_synonym_lst_ , all_anotnym_lst_, fwd_subset_,
        num_steps,batch_size_,attract_margin_value,repel_margin_value,regularisation_constant_value):
    loss_ = []
    with tf.Session(graph=graph) as session:
      # We must initialize all variables before we use them.
        
        init.run()
        print('Initialized')
        current_iteration=0
        average_loss = 0
        
        synonyms_left_ = [ seq[0] for seq in all_synonym_lst_ ]
        synonyms_right_ = [ seq[1] for seq in all_synonym_lst_ ]
        attract_indices_left_ = [fwd_subset_[w] for w in synonyms_left_]
        attract_indices_right_ = [fwd_subset_[w] for w in synonyms_right_]
        
#         negative_attract_left_ = [ seq[0] for seq in negative_examples_synonyms_ ]
#         negative_attract_right_ = [ seq[1] for seq in negative_examples_synonyms_ ]
#         negative_attract_indices_left_ = [fwd_subset_[w] for w in negative_attract_left_]
#         negative_attract_indices_right_ = [fwd_subset_[w] for w in negative_attract_right_]
        
        antonym_left_ = [ seq[0] for seq in all_anotnym_lst_ ]
        antonym_right_ = [ seq[1] for seq in all_anotnym_lst_ ]
        repel_indices_left_ = [fwd_subset_[w] for w in antonym_left_]
        repel_indices_right_ = [fwd_subset_[w] for w in antonym_right_]
        
#         negative_repel_left_ = [ seq[0] for seq in negative_examples_antonyms_ ]
#         negative_repel_right_ = [ seq[1] for seq in negative_examples_antonyms_ ]
#         negative_repel_indices_left_ = [fwd_subset_[w] for w in negative_repel_left_]
#         negative_repel_indices_right_ = [fwd_subset_[w] for w in negative_repel_right_]
        
        syn_cnt = len(synonyms_left_)
        ant_cnt =  len(antonym_left_)
        
        syn_batch_ = int(syn_cnt/batch_size_)
        ant_batch_ = int(ant_cnt/batch_size_)
        
        batch_per_epoch_ =  syn_batch_+ant_batch_
        last_time = time.time()
        average_loss = []
        while current_iteration < int(num_steps):

            # how many attract/repel batches we've done in this epoch so far.
            antonym_counter = 0
            synonym_counter = 0

            order_of_synonyms = list(range(0, syn_cnt))
            order_of_antonyms = list(range(0, ant_cnt))

            random.shuffle(order_of_synonyms)
            random.shuffle(order_of_antonyms)

            # list of 0 where we run synonym batch, 1 where we run antonym batch
            list_of_batch_types = [0] * batch_per_epoch_
            list_of_batch_types[syn_batch_:] = [1] * ant_batch_ # all antonym batches to 1
            random.shuffle(list_of_batch_types)
        
            if current_iteration == 0:
                print("\nStarting epoch:", current_iteration+1, "\n")
            else:
                print("\nStarting epoch:", current_iteration+1, "Last epoch took:", round(time.time() - last_time, 1), "seconds. \n")
                last_time = time.time()
                
            for batch_index in range(0, batch_per_epoch_):
                syn_or_ant_batch = list_of_batch_types[batch_index]

                if syn_or_ant_batch == 0:
                    # do one synonymy batch:
                    synonymy_examples_lt = [attract_indices_left_[order_of_synonyms[int(x)]] 
                                            for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
                    synonymy_examples_rt = [attract_indices_right_[order_of_synonyms[int(x)]] 
                                            for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
#                     current_negatives_syn_lt = [negative_attract_indices_left_[order_of_synonyms[int(x)]] 
#                                             for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
                    
                    current_negatives_syn_lt = extract_negative(W_dynamic.eval(), synonymy_examples_lt, syn_indicator_ = True)
                    current_negatives_syn_rt = extract_negative(W_dynamic.eval(), synonymy_examples_rt, syn_indicator_ = True)
#                     current_negatives_syn_rt = [negative_attract_indices_right_[order_of_synonyms[int(x)]]
#                                             for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]

                    loss_val = session.run([attract_cost_step],
                                  feed_dict={attract_examples_lt: synonymy_examples_lt,
                                             attract_examples_rt: synonymy_examples_rt,
                                             negative_examples_attract_lt: current_negatives_syn_lt,
                                             negative_examples_attract_rt: current_negatives_syn_rt,
                                             attract_margin: attract_margin_value, 
                                             regularisation_constant: regularisation_constant_value})
                    synonym_counter += 1

                else:
                    
                    antonym_examples_lt = [repel_indices_left_[order_of_antonyms[int(x)]] 
                                            for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
                    antonym_examples_rt = [repel_indices_right_[order_of_antonyms[int(x)]] 
                                            for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
#                     current_negatives_ant_lt = [negative_repel_indices_left_[order_of_antonyms[int(x)]] 
#                                             for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
#                     current_negatives_ant_rt = [negative_repel_indices_right_[order_of_antonyms[int(x)]]
#                                             for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
                    
                    current_negatives_ant_lt = extract_negative(W_dynamic.eval(), antonym_examples_lt, syn_indicator_ = False)
                    current_negatives_ant_rt = extract_negative(W_dynamic.eval(), antonym_examples_rt, syn_indicator_ = False)
                    
                    loss_val = session.run([repel_cost_step],
                                  feed_dict={repel_examples_lt: antonym_examples_lt,
                                             repel_examples_rt: antonym_examples_rt,
                                             negative_examples_repel_lt: current_negatives_ant_lt,
                                             negative_examples_repel_rt: current_negatives_ant_rt,
                                             repel_margin: repel_margin_value, 
                                             regularisation_constant: regularisation_constant_value})
                    
                    antonym_counter += 1
                average_loss.append(loss_val)
                loss_.append(average_loss)
            current_iteration += 1
#                     antonymy_examples = [self.antonyms[order_of_antonyms[x]] 
#                                          for x in range(antonym_counter * self.batch_size, (antonym_counter+1) * self.batch_size)]
                    
                    
#                     current_negatives = self.extract_negative_examples(antonymy_examples, attract_batch=False)

#                     self.sess.run([self.repel_cost_step], feed_dict={self.repel_examples: antonymy_examples, self.negative_examples_repel: current_negatives, \
#                                                                   self.repel_margin: self.repel_margin_value, self.regularisation_constant: self.regularisation_constant_value})
        
        final_embeddings = W_dynamic.eval()
        initial_embeddings = W_init.eval()
        return final_embeddings, initial_embeddings,loss_

In [38]:
f_embeddings,i_embeddings, loss = run(graph, all_synonym_lst, all_antonym_lst, fwd_subset,5.0,10.0,0.6,0.0, 0.000000001)

Initialized

Starting epoch: 1 


Starting epoch: 2 Last epoch took: 92.3 seconds. 


Starting epoch: 3 Last epoch took: 36.3 seconds. 


Starting epoch: 4 Last epoch took: 31.3 seconds. 


Starting epoch: 5 Last epoch took: 34.2 seconds. 



In [42]:
np.shape(f_embeddings)

(400000, 100)

In [43]:
np.shape(i_embeddings)

(400000, 100)

In [44]:
np.shape(embeddings_google_scaled)

(400000, 100)

In [45]:
f_embeddings[vocab_dict['software']]

array([-0.04303241, -0.02462361,  0.09771127,  0.0373629 ,  0.2545208 ,
       -0.20046322,  0.01434526, -0.22431813,  0.17752431, -0.05215071,
        0.16194609, -0.00568303, -0.01110241, -0.23066767, -0.06362169,
        0.06779336,  0.1010457 ,  0.1491268 ,  0.08973411,  0.20283034,
       -0.26765427, -0.11307473,  0.07173959,  0.24790318,  0.07961931,
       -0.08651975, -0.03359356,  0.11608226, -0.15911503,  0.0277115 ,
        0.21371303,  0.30736616, -0.1305928 , -0.1190127 ,  0.14981721,
        0.05696556, -0.10413086, -0.16227709,  0.23613481, -0.13731743,
        0.06654156,  0.01755162, -0.08489048,  0.03719721, -0.23736516,
       -0.03598461,  0.02236842,  0.0324764 ,  0.06239328,  0.01444592,
       -0.0507765 ,  0.13192989, -0.01740786, -0.08505104,  0.11261337,
       -0.28377438, -0.19898446,  0.07600551,  0.49270594,  0.08079895,
       -0.00872659, -0.05802275, -0.13353747,  0.13339959, -0.14529584,
        0.053238  ,  0.23131913, -0.04504349,  0.25668713,  0.00

In [46]:
i_embeddings[vocab_dict['software']]

array([-7.45223239e-02,  1.50411287e-02,  6.52041361e-02,  1.98040288e-02,
        1.31744072e-01, -6.70875162e-02,  4.14898954e-02, -1.28355220e-01,
        1.02836266e-01,  4.74901823e-03,  1.03906758e-01, -6.29567280e-02,
        1.85546130e-02, -1.70985013e-01, -7.01478198e-02,  7.88412914e-02,
        1.54341549e-01,  8.48739669e-02, -1.31275160e-02,  1.45644993e-01,
       -1.84944555e-01, -1.14446588e-01,  1.08426251e-01,  1.17665768e-01,
        2.18601674e-02, -4.96573783e-02,  2.16874089e-02,  3.40412110e-02,
       -1.56485617e-01, -2.59354264e-02,  9.68853477e-03,  2.20606923e-01,
       -1.67144224e-01, -5.85544631e-02,  1.19339362e-01,  4.71215248e-02,
       -5.85405827e-02, -4.55728695e-02,  1.10530213e-01, -1.53758481e-01,
       -1.31566689e-04,  3.28889713e-02, -5.92146516e-02, -5.45532443e-04,
       -1.07480705e-01, -2.40181107e-02, -2.23337114e-02, -2.18833070e-02,
        6.48802146e-02,  7.80777587e-03,  2.13588588e-02,  5.41414022e-02,
       -2.10148841e-02,  

In [47]:
embeddings_google_scaled[vocab_dict['software']]

array([-7.45223239e-02,  1.50411287e-02,  6.52041361e-02,  1.98040288e-02,
        1.31744072e-01, -6.70875162e-02,  4.14898954e-02, -1.28355220e-01,
        1.02836266e-01,  4.74901823e-03,  1.03906758e-01, -6.29567280e-02,
        1.85546130e-02, -1.70985013e-01, -7.01478198e-02,  7.88412914e-02,
        1.54341549e-01,  8.48739669e-02, -1.31275160e-02,  1.45644993e-01,
       -1.84944555e-01, -1.14446588e-01,  1.08426251e-01,  1.17665768e-01,
        2.18601674e-02, -4.96573783e-02,  2.16874089e-02,  3.40412110e-02,
       -1.56485617e-01, -2.59354264e-02,  9.68853477e-03,  2.20606923e-01,
       -1.67144224e-01, -5.85544631e-02,  1.19339362e-01,  4.71215248e-02,
       -5.85405827e-02, -4.55728695e-02,  1.10530213e-01, -1.53758481e-01,
       -1.31566689e-04,  3.28889713e-02, -5.92146516e-02, -5.45532443e-04,
       -1.07480705e-01, -2.40181107e-02, -2.23337114e-02, -2.18833070e-02,
        6.48802146e-02,  7.80777587e-03,  2.13588588e-02,  5.41414022e-02,
       -2.10148841e-02,  

In [48]:
#Saving the generated word embeddings
np.save('Embeddings_eqpt_taxo_competetive_with_Google_6_AUG_corrected', f_embeddings)

#Saving the dictionary
import json
json = json.dumps(vocab_dict)
f = open("word_index_dictionary_eqpt_taxo_competetive_with_Google_6_AUG_corrected.json","w")
f.write(json)
f.close()

In [18]:
def random_different_from(top_range, number_to_not_repeat):

    result = random.randint(0, top_range-1)
    while result == number_to_not_repeat:
        result = random.randint(0, top_range-1)

    return result
def mix_sampling(list_of_examples, negative_examples):
    """
    Converts half of the negative examples to random words from the batch (that are not in the given example pair).  
    """
    mixed_negative_examples = []
    batch_size = len(list_of_examples)

    for idx, (left_idx, right_idx) in enumerate(negative_examples):

        new_left = left_idx
        new_right = right_idx

        if random.random() >= 0.5:
            new_left = list_of_examples[random_different_from(batch_size, idx)][random.randint(0, 1)]
        
        if random.random() >= 0.5:
            new_right = list_of_examples[random_different_from(batch_size, idx)][random.randint(0, 1)]

        mixed_negative_examples.append((new_left, new_right))

    return mixed_negative_examples

In [ ]:
graph = tf.Graph()
with graph.as_default():
    """
    Initialises the TensorFlow Attract-Repel model.
    """
    attract_examples_lt = tf.placeholder(tf.int32, shape=(None)) # each element is the position of word vector.
    attract_examples_rt = tf.placeholder(tf.int32, shape=(None)) # each element is the position of word vector.
    
    repel_examples_lt = tf.placeholder(tf.int32, shape=(None)) # each element is again the position of word vector.
    repel_examples_rt = tf.placeholder(tf.int32, shape=(None)) # each element is again the position of word vector.
    
    negative_examples_attract_lt = tf.placeholder(tf.int32, shape=(None))
    negative_examples_attract_rt = tf.placeholder(tf.int32, shape=(None))
    
    negative_examples_repel_lt = tf.placeholder(tf.int32, shape=(None))
    negative_examples_repel_rt = tf.placeholder(tf.int32, shape=(None))
    
    attract_margin = tf.placeholder("float")
    repel_margin = tf.placeholder("float")
    regularisation_constant = tf.placeholder("float")

    # Initial (distributional) vectors. Needed for L2 regularisation.         
    W_init = tf.constant(embeddings_google_scaled, name="W_init")

    # Variable storing the updated word vectors. 
    W_dynamic = tf.Variable(embeddings_google_scaled, name="W_dynamic")
    
    # Attract Cost Function: 

    # placeholders for example pairs...
    attract_examples_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, attract_examples_lt), 1) 
    attract_examples_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, attract_examples_rt), 1)

    # and their respective negative examples:
    negative_examples_attract_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_attract_lt), 1)
    negative_examples_attract_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_attract_rt), 1)

    # dot product between the example pairs. 
    attract_similarity_between_examples = tf.reduce_sum(tf.multiply(attract_examples_left, attract_examples_right), 1) 

    # dot product of each word in the example with its negative example. 
    attract_similarity_to_negatives_left = tf.reduce_sum(tf.multiply(attract_examples_left, negative_examples_attract_left), 1) 
    attract_similarity_to_negatives_right = tf.reduce_sum(tf.multiply(attract_examples_right, negative_examples_attract_right), 1)

    # and the final Attract Cost Function (sans regularisation):
    attract_cost = tf.nn.relu(attract_margin + attract_similarity_to_negatives_left - attract_similarity_between_examples) + \
                   tf.nn.relu(attract_margin + attract_similarity_to_negatives_right - attract_similarity_between_examples)

    # Repel Cost Function: 

    # placeholders for example pairs...
    repel_examples_left = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, repel_examples_lt), 1) # becomes batch_size X vector_dimension 
    repel_examples_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, repel_examples_rt), 1)

    # and their respective negative examples:
    negative_examples_repel_left  = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_repel_lt), 1)
    negative_examples_repel_right = tf.nn.l2_normalize(tf.nn.embedding_lookup(W_dynamic, negative_examples_repel_rt), 1)

    # dot product between the example pairs. 
    repel_similarity_between_examples = tf.reduce_sum(tf.multiply(repel_examples_left, repel_examples_right), 1) # becomes batch_size again, might need tf.squeeze

    # dot product of each word in the example with its negative example. 
    repel_similarity_to_negatives_left = tf.reduce_sum(tf.multiply(repel_examples_left, negative_examples_repel_left), 1)
    repel_similarity_to_negatives_right = tf.reduce_sum(tf.multiply(repel_examples_right, negative_examples_repel_right), 1)

    # and the final Repel Cost Function (sans regularisation):
    repel_cost = tf.nn.relu(repel_margin - repel_similarity_to_negatives_left + repel_similarity_between_examples) + \
                   tf.nn.relu(repel_margin - repel_similarity_to_negatives_right + repel_similarity_between_examples)

    # The Regularisation Cost (separate for the two terms, depending on which one is called): 

    # load the original distributional vectors for the example pairs: 
    original_attract_examples_left = tf.nn.embedding_lookup(W_init, attract_examples_lt)
    original_attract_examples_right = tf.nn.embedding_lookup(W_init, attract_examples_rt)

    original_repel_examples_left = tf.nn.embedding_lookup(W_init, repel_examples_lt)
    original_repel_examples_right = tf.nn.embedding_lookup(W_init, repel_examples_rt)

    # and then define the respective regularisation costs:
    regularisation_cost_attract = regularisation_constant * (tf.nn.l2_loss(original_attract_examples_left - attract_examples_left) + tf.nn.l2_loss(original_attract_examples_right - attract_examples_right))
    attract_cost += regularisation_cost_attract

    regularisation_cost_repel = regularisation_constant * (tf.nn.l2_loss(original_repel_examples_left - repel_examples_left) + tf.nn.l2_loss(original_repel_examples_right - repel_examples_right))
    repel_cost += regularisation_cost_repel

    # Finally, we define the training step functions for both steps. 

    tvars = tf.trainable_variables()
    attract_grads = [tf.clip_by_value(grad, -2., 2.) for grad in tf.gradients(attract_cost, tvars)]
    repel_grads = [tf.clip_by_value(grad, -2., 2.) for grad in tf.gradients(repel_cost, tvars)]

    attract_optimiser = tf.train.AdagradOptimizer(0.05) 
    repel_optimiser = tf.train.AdagradOptimizer(0.05) 

    attract_cost_step = attract_optimiser.apply_gradients(zip(attract_grads, tvars))
    repel_cost_step = repel_optimiser.apply_gradients(zip(repel_grads, tvars))
    init = tf.global_variables_initializer()
    

In [20]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
def extract_negative_examples(list_left_batch,list_right_batch,graph, attract_batch = True):
    """
    For each example in the minibatch, this method returns the closest vector which is not 
    in each words example pair. 
    """
    with tf.Session(graph=graph) as session:
        init.run()
        list_of_representations = []
        list_of_indices = []

        representations = session.run([attract_examples_left, attract_examples_right], feed_dict={attract_examples_lt: list_left_batch,
                                                                                                 attract_examples_rt:list_right_batch})
        
        
        list_minibatch = []
        for i in range(len(list_left_batch)):
            list_minibatch.append((list_left_batch[i],list_right_batch[i]))
        
        for idx, (example_left, example_right) in enumerate(list_minibatch):

            list_of_representations.append(representations[0][idx])
            list_of_representations.append(representations[1][idx])

            list_of_indices.append(example_left)
            list_of_indices.append(example_right)

        condensed_distance_list = pdist(list_of_representations, 'cosine') 
        square_distance_list = squareform(condensed_distance_list)   

        if attract_batch: 
            default_value = 2.0 # value to set for given attract/repel pair, so that it can not be found as closest or furthest away. 
        else:
            default_value = 0.0 # for antonyms, we want the opposite value from the synonym one. Cosine Distance is [0,2]. 

        for i in range(len(square_distance_list)):

            square_distance_list[i,i]=default_value 

            if i % 2 == 0:
                square_distance_list[i,i+1] = default_value 
            else:
                square_distance_list[i,i-1] = default_value

        if attract_batch:
            negative_example_indices = np.argmin(square_distance_list,axis=1) # for each of the 100 elements, finds the index which has the minimal cosine distance (i.e. most similar). 
        else:
            negative_example_indices = np.argmax(square_distance_list, axis=1) # for antonyms, find the least similar one. 

        negative_examples = []

        for idx in range(len(list_minibatch)):

            negative_example_left = list_of_indices[negative_example_indices[2 * idx]] 
            negative_example_right = list_of_indices[negative_example_indices[2 * idx + 1]]

            negative_examples.append((negative_example_left, negative_example_right))            

        negative_examples = mix_sampling(list_minibatch, negative_examples)
        
        neg_left_=[]
        neg_right_=[]
        for a,b in negative_examples:
            neg_left_.append(a)
            neg_right_.append(b)
        
        return neg_left_,neg_right_

In [21]:
import time
import random
def run(graph, all_synonym_lst_ , all_anotnym_lst_, fwd_subset_,
        num_steps,batch_size_,attract_margin_value,repel_margin_value,regularisation_constant_value):
    loss_ = []
    with tf.Session(graph=graph) as session:
      # We must initialize all variables before we use them.
        
        init.run()
        print('Initialized')
        current_iteration=0
        average_loss = 0
        
        synonyms_left_ = [ seq[0] for seq in all_synonym_lst_ ]
        synonyms_right_ = [ seq[1] for seq in all_synonym_lst_ ]
        attract_indices_left_ = [fwd_subset_[w] for w in synonyms_left_]
        attract_indices_right_ = [fwd_subset_[w] for w in synonyms_right_]
        
#         negative_attract_left_ = [ seq[0] for seq in negative_examples_synonyms_ ]
#         negative_attract_right_ = [ seq[1] for seq in negative_examples_synonyms_ ]
#         negative_attract_indices_left_ = [fwd_subset_[w] for w in negative_attract_left_]
#         negative_attract_indices_right_ = [fwd_subset_[w] for w in negative_attract_right_]
        
        antonym_left_ = [ seq[0] for seq in all_anotnym_lst_ ]
        antonym_right_ = [ seq[1] for seq in all_anotnym_lst_ ]
        repel_indices_left_ = [fwd_subset_[w] for w in antonym_left_]
        repel_indices_right_ = [fwd_subset_[w] for w in antonym_right_]
        
#         negative_repel_left_ = [ seq[0] for seq in negative_examples_antonyms_ ]
#         negative_repel_right_ = [ seq[1] for seq in negative_examples_antonyms_ ]
#         negative_repel_indices_left_ = [fwd_subset_[w] for w in negative_repel_left_]
#         negative_repel_indices_right_ = [fwd_subset_[w] for w in negative_repel_right_]
        
        syn_cnt = len(synonyms_left_)
        ant_cnt =  len(antonym_left_)
        
        syn_batch_ = int(syn_cnt/batch_size_)
        ant_batch_ = int(ant_cnt/batch_size_)
        
        batch_per_epoch_ =  syn_batch_+ant_batch_
        last_time = time.time()
        average_loss = []
        while current_iteration < int(num_steps):

            # how many attract/repel batches we've done in this epoch so far.
            antonym_counter = 0
            synonym_counter = 0

            order_of_synonyms = list(range(0, syn_cnt))
            order_of_antonyms = list(range(0, ant_cnt))

            random.shuffle(order_of_synonyms)
            random.shuffle(order_of_antonyms)

            # list of 0 where we run synonym batch, 1 where we run antonym batch
            list_of_batch_types = [0] * batch_per_epoch_
            list_of_batch_types[syn_batch_:] = [1] * ant_batch_ # all antonym batches to 1
            random.shuffle(list_of_batch_types)
        
            if current_iteration == 0:
                print("\nStarting epoch:", current_iteration+1, "\n")
            else:
                print("\nStarting epoch:", current_iteration+1, "Last epoch took:", round(time.time() - last_time, 1), "seconds. \n")
                last_time = time.time()
                
            for batch_index in range(0, batch_per_epoch_):
                syn_or_ant_batch = list_of_batch_types[batch_index]

                if syn_or_ant_batch == 0:
                    # do one synonymy batch:
                    synonymy_examples_lt = [attract_indices_left_[order_of_synonyms[int(x)]] 
                                            for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
                    synonymy_examples_rt = [attract_indices_right_[order_of_synonyms[int(x)]] 
                                            for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
#                     current_negatives_syn_lt = [negative_attract_indices_left_[order_of_synonyms[int(x)]] 
#                                             for x in range(int(synonym_counter * batch_size_),int((synonym_counter+1) * batch_size_))]
                    
                    current_negatives_syn_lt, current_negatives_syn_rt = extract_negative_examples(synonymy_examples_lt,synonymy_examples_rt,
                                                                                graph=graph,attract_batch = True)
                    loss_val = session.run([attract_cost_step],
                                  feed_dict={attract_examples_lt: synonymy_examples_lt,
                                             attract_examples_rt: synonymy_examples_rt,
                                             negative_examples_attract_lt: current_negatives_syn_lt,
                                             negative_examples_attract_rt: current_negatives_syn_rt,
                                             attract_margin: attract_margin_value, 
                                             regularisation_constant: regularisation_constant_value})
                    synonym_counter += 1

                else:
                    
                    antonym_examples_lt = [repel_indices_left_[order_of_antonyms[int(x)]] 
                                            for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
                    antonym_examples_rt = [repel_indices_right_[order_of_antonyms[int(x)]] 
                                            for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
#                     current_negatives_ant_lt = [negative_repel_indices_left_[order_of_antonyms[int(x)]] 
#                                             for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
#                     current_negatives_ant_rt = [negative_repel_indices_right_[order_of_antonyms[int(x)]]
#                                             for x in range(int(antonym_counter * batch_size_),int((antonym_counter+1) * batch_size_))]
                    
                    current_negatives_ant_lt,current_negatives_ant_rt = extract_negative_examples(antonym_examples_lt,antonym_examples_rt,
                                                                                graph=graph,attract_batch = False)
                    
                    loss_val = session.run([repel_cost_step],
                                  feed_dict={repel_examples_lt: antonym_examples_lt,
                                             repel_examples_rt: antonym_examples_rt,
                                             negative_examples_repel_lt: current_negatives_ant_lt,
                                             negative_examples_repel_rt: current_negatives_ant_rt,
                                             repel_margin: repel_margin_value, 
                                             regularisation_constant: regularisation_constant_value})
                    
                    antonym_counter += 1
                average_loss.append(loss_val)
                loss_.append(average_loss)
            current_iteration += 1
#                     antonymy_examples = [self.antonyms[order_of_antonyms[x]] 
#                                          for x in range(antonym_counter * self.batch_size, (antonym_counter+1) * self.batch_size)]
                    
                    
#                     current_negatives = self.extract_negative_examples(antonymy_examples, attract_batch=False)

#                     self.sess.run([self.repel_cost_step], feed_dict={self.repel_examples: antonymy_examples, self.negative_examples_repel: current_negatives, \
#                                                                   self.repel_margin: self.repel_margin_value, self.regularisation_constant: self.regularisation_constant_value})
        
        final_embeddings = W_dynamic.eval()
        initial_embeddings = W_init.eval()
        return final_embeddings, initial_embeddings,loss_

In [22]:
f_embeddings,i_embeddings, loss = run(graph, all_synonym_lst, all_antonym_lst, fwd_subset,5.0,10.0,0.6,0.0, 0.000000001)

Initialized

Starting epoch: 1 


Starting epoch: 2 Last epoch took: 860.3 seconds. 


Starting epoch: 3 Last epoch took: 731.7 seconds. 


Starting epoch: 4 Last epoch took: 807.8 seconds. 


Starting epoch: 5 Last epoch took: 679.2 seconds. 



In [24]:
from scipy.spatial.distance import cosine
print(cosine(f_embeddings[dictionary['piston']],f_embeddings[dictionary['liner']]))
print(cosine(i_embeddings[dictionary['piston']],i_embeddings[dictionary['liner']]))
print(cosine(f_embeddings[dictionary['module']],f_embeddings[dictionary['spring']]))
print(cosine(i_embeddings[dictionary['module']],i_embeddings[dictionary['spring']]))

0.43642961978912354
0.43642961978912354
0.3400229811668396
0.3400229811668396


In [25]:
#Saving the generated word embeddings
np.save('Embeddings_eqpt_taxo_competetive_with_Google_10_AUG_updated_neg_sam', f_embeddings)

#Saving the dictionary
import json
json = json.dumps(vocab_dict)
f = open("word_index_dictionary_eqpt_taxo_competetive_with_Google_10_AUG_updated_neg_sam.json","w")
f.write(json)
f.close()